In [180]:
import ipywidgets as widgets
import pandas as pd
import io
from ipywidgets import GridspecLayout, VBox
from ipywidgets import Button, Layout, jslink, IntText, IntSlider


In [181]:
def on_click_getdetails(change):
    rogr_data = pd.read_excel(io.BytesIO(btn_upload.data[-1]),  index_col = None, header = None)
    rogr_data = rogr_data.drop(columns=[4])
    rogr_data = rogr_data.dropna()
    rogr_data.columns = rogr_data.iloc[0]
    rogr_data = rogr_data[2:]
    rogr_data['Date'] = pd.to_datetime(rogr_data['Date'], dayfirst=True,errors='coerce')
    rogr_data = rogr_data[rogr_data['Date'].notna()]
    rogr_data['flat_no'] = rogr_data['Narration'].str.extract(r'(-ROGRWA[OT])(?P<flat_no>[G 0-9]{3})(-)', expand=True)['flat_no']
    rogr_data.sort_values(['Date'], ascending=True).reset_index(drop=True, inplace=True)
    paid_flats = rogr_data[((rogr_data['Date'].dt.month == 7) & (rogr_data['Deposit Amt.'] >= 2500))]['flat_no']
    display_flat_payment_details(paid_flats)


In [182]:
btn_upload = widgets.FileUpload(multiple=False)
btn_run = widgets.Button(description='Get Details')
btn_run.on_click(on_click_getdetails)
out_pl = widgets.Output()
out_pl.clear_output()

In [183]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

def display_flat_payment_details(paid_flats):
    out_pl.clear_output()
    grid = GridspecLayout(4, 12)

    for i in range(4):
        if i == 0:
            floor_no = "G"
        else:
            floor_no = str(i)
        for j in range(1,13):
            flat_no = floor_no + str(j).zfill(2)
            color = 'danger'
            if flat_no in list(paid_flats):
                color = 'success'
            grid[i, j-1] = create_expanded_button('{}'.format(flat_no), color)
    grid
    with out_pl: display(grid)

In [184]:
VBox([widgets.Label('ROGR PAYMENT DETAILS'), 
      btn_upload, btn_run, out_pl])